In [15]:
# !pip install -r ./requirements.txt

In [2]:
# !pip uninstall virny -y

In [4]:
# Install using an HTTP link
# !pip install git+https://github.com/DataResponsibly/Virny.git@development

# Install using an SSH link
# !pip install git+ssh://git@github.com/DataResponsibly/Virny.git@development

In [5]:
# !pip install aif360

In [6]:
# !pip install BlackBoxAuditing==0.1.54

In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [4]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

In [5]:
cur_folder_name = os.getcwd().split('/')[-1]
if cur_folder_name != "fairness-variance":
    os.chdir("../..")

print('Current location: ', os.getcwd())

Current location:  /home/denys_herasymuk/UCU/4course_2term/Bachelor_Thesis/Code/fairness-variance


## Import dependencies

In [6]:
import os
import copy

from virny.utils.custom_initializers import create_config_obj
from virny.datasets import LawSchoolDataset

from configs.constants import TEST_SET_FRACTION, EXPERIMENT_SEEDS
from configs.models_config_for_tuning import get_model_params_for_mult_repair_levels

from source.preprocessing import get_simple_preprocessor
from source.experiment_interface import run_exp_iter_with_disparate_impact

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'


## Define Input Variables

In [8]:
# ROOT_DIR = os.path.join(os.getcwd(), "..", "..")
ROOT_DIR = os.getcwd()
EXPERIMENT_NAME = 'mult_repair_levels_law_school'
DB_COLLECTION_NAME = 'exp_mult_repair_levels'
SAVE_RESULTS_DIR_PATH = os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME)
# FAIR_INTERVENTION_PARAMS_LST = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
FAIR_INTERVENTION_PARAMS_LST = [0.0, 0.2, 0.4]

config_yaml_path = os.path.join(ROOT_DIR, 'notebooks', EXPERIMENT_NAME, 'law_school_2018_config.yaml')
metrics_computation_config = create_config_obj(config_yaml_path=config_yaml_path)

## Define a db writer and custom fields to insert into your database

In [9]:
import os
from dotenv import load_dotenv

load_dotenv('./configs/secrets.env')
os.getenv("DB_NAME")

'fairness_variance'

In [10]:
from source.utils.db_functions import connect_to_mongodb

client, collection_obj, db_writer_func = connect_to_mongodb(DB_COLLECTION_NAME)

In [12]:
import uuid

custom_table_fields_dct = {
    # 'session_uuid': str(uuid.uuid4()),
    'session_uuid': '4f9ef0c2-e2d6-46ee-9868-26fd5c2ef2f4',
}
print('Current session uuid: ', custom_table_fields_dct['session_uuid'])

Current session uuid:  4f9ef0c2-e2d6-46ee-9868-26fd5c2ef2f4


## Initialize custom objects

In [13]:
data_loader = LawSchoolDataset()
data_loader.X_data.head()

,decile1b,decile3,lsat,ugpa,zfygpa,zgpa,fulltime,fam_inc,male,tier,race
0,10.0,10.0,44.0,3.5,1.33,1.88,1.0,5.0,0.0,4.0,White
1,5.0,4.0,29.0,3.5,-0.11,-0.57,1.0,4.0,0.0,2.0,White
2,8.0,7.0,37.0,3.4,0.63,0.37,1.0,3.0,1.0,4.0,White
3,8.0,7.0,43.0,3.3,0.67,0.34,1.0,4.0,0.0,4.0,White
4,3.0,2.0,41.0,3.3,-0.67,-1.30,1.0,4.0,0.0,5.0,White


In [14]:
data_loader.X_data.shape

(20798, 11)

In [16]:
data_loader.X_data.dtypes

decile1b    float64
decile3     float64
lsat        float64
ugpa        float64
zfygpa      float64
zgpa        float64
fulltime     object
fam_inc      object
male         object
tier         object
race         object
dtype: object

In [17]:
data_loader.y_data.value_counts()

1.0    18505
0.0     2293
Name: pass_bar, dtype: int64

In [19]:
data_loader.X_data['race'].value_counts()

White        17491
Non-White     3307
Name: race, dtype: int64

In [20]:
data_loader.X_data['male'].value_counts()

1.0    11675
0.0     9123
Name: male, dtype: int64

## Run experiment iterations

### Experiment iteration 1

In [31]:
# tuned_params_filenames = ['tuning_results_Folktables_NY_2018_Employment_alpha_0.8_20230706__115508.csv']
# tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
#                          for tuned_params_filename in tuned_params_filenames]

In [32]:
# Configs for an experiment iteration
exp_iter_num = 1
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [33]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
                                   with_tuning=True,
                                   # with_tuning=False,
                                   # tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)

2023-07-31 13:25:11 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 100,
 'experiment_iteration': 'Exp_iter_1',
 'fair_intervention_params_lst': '[0.0, 0.1]',
 'model_init_seed': 100,
 'session_uuid': '116104d6-7d63-4564-ae57-82b19b260fa5'}




2023-07-31 13:25:12 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([10155, 11689, 12599, 12193,  8678,  8217,  4670, 12087,  5235,
             4189,  7278, 10642,  5284,  7002, 14642, 10594,  7701,  8686,
             8665,  6253],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([10155, 11689, 12599, 12193,  8678,  8217,  4670, 12087,  5235,
             4189,  7278, 10642,  5284,  7002, 14642, 10594,  7701,  8686,
             8665,  6253],
           dtype='int64')


Multiple alphas:   0%|          | 0/2 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

intervention_param:  0.1
2023/07/31, 13:35:55: Tuning RandomForestClassifier...
2023/07/31, 13:41:51: Tuning for RandomForestClassifier is finished [F1 score = 0.6658729175624925, Accuracy = 0.7133333333333334]



Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 2

In [44]:
# Configs for an experiment iteration
exp_iter_num = 2
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.0_20230731__173053.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.1_20230731__174151.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.2_20230731__173153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.3_20230731__174153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.4_20230731__173254.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [45]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)

2023-07-31 14:43:04 experiment_interface.py INFO    : Start an experiment iteration for the following custom params:
INFO:root:Start an experiment iteration for the following custom params:


{'dataset_split_seed': 200,
 'experiment_iteration': 'Exp_iter_2',
 'fair_intervention_params_lst': '[0.0, 0.1, 0.2, 0.3, 0.4]',
 'model_init_seed': 200,
 'session_uuid': '116104d6-7d63-4564-ae57-82b19b260fa5'}




2023-07-31 14:43:04 experiment_interface.py INFO    : The dataset is preprocessed
INFO:root:The dataset is preprocessed


Top indexes of an X_test in a base flow dataset:  Int64Index([ 6043,  3745,  5159,  7241,  7820,  3695, 11501, 11432,  1163,
             8994,  7972,  2554,  9884,  2008,  6884, 11995,  5200,  4649,
            10244, 13775],
           dtype='int64')
Top indexes of an y_test in a base flow dataset:  Int64Index([ 6043,  3745,  5159,  7241,  7820,  3695, 11501, 11432,  1163,
             8994,  7972,  2554,  9884,  2008,  6884, 11995,  5200,  4649,
            10244, 13775],
           dtype='int64')


Multiple alphas:   0%|          | 0/5 [00:00<?, ?it/s]

...

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

intervention_param:  0.4
Path for tuned params:  /home/dh3553/projects/fairness-variance/results/mult_repair_levels_pubcov_CA/tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.4_20230731__173254.csv
RandomForestClassifier:  {'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': 40, 'max_features': 'sqrt', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 10, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 201, 'verbose': 0, 'warm_start': False}


Multiple runs progress:   0%|          | 0/1 [00:00<?, ?it/s]

Analyze models in one run:   0%|          | 0/1 [00:00<?, ?it/s]

Classifiers testing by bootstrap:   0%|          | 0/200 [00:00<?, ?it/s]

### Experiment iteration 3

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 3
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.0_20230731__173053.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.1_20230731__174151.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.2_20230731__173153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.3_20230731__174153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.4_20230731__173254.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)

### Experiment iteration 4

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 4
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.0_20230731__173053.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.1_20230731__174151.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.2_20230731__173153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.3_20230731__174153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.4_20230731__173254.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [46]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)

### Experiment iteration 5

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 5
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.0_20230731__173053.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.1_20230731__174151.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.2_20230731__173153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.3_20230731__174153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.4_20230731__173254.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)

### Experiment iteration 6

In [ ]:
# Configs for an experiment iteration
exp_iter_num = 6
experiment_seed = EXPERIMENT_SEEDS[exp_iter_num - 1]
tuned_params_filenames = [
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.0_20230731__173053.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.1_20230731__174151.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.2_20230731__173153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.3_20230731__174153.csv',
    'tuning_results_Folktables_CA_2018_Public_Coverage_alpha_0.4_20230731__173254.csv',
]
tuned_params_df_paths = [os.path.join(ROOT_DIR, 'results', EXPERIMENT_NAME, tuned_params_filename)
                         for tuned_params_filename in tuned_params_filenames]
custom_table_fields_dct['experiment_iteration'] = f'Exp_iter_{exp_iter_num}'

exp_iter_data_loader = copy.deepcopy(data_loader)  # Add deepcopy to avoid data leakage
models_params_for_tuning = get_model_params_for_mult_repair_levels(experiment_seed)

In [ ]:
run_exp_iter_with_disparate_impact(data_loader=exp_iter_data_loader,
                                   experiment_seed=experiment_seed,
                                   test_set_fraction=TEST_SET_FRACTION,
                                   db_writer_func=db_writer_func,
                                   fair_intervention_params_lst=FAIR_INTERVENTION_PARAMS_LST,
                                   models_params_for_tuning=models_params_for_tuning,
                                   metrics_computation_config=metrics_computation_config,
                                   custom_table_fields_dct=custom_table_fields_dct,
#                                    with_tuning=True,
                                   with_tuning=False,
                                   tuned_params_df_paths=tuned_params_df_paths,
                                   save_results_dir_path=SAVE_RESULTS_DIR_PATH,
                                   verbose=True)